In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('use_inf_as_na', True)

/tmp/ipykernel_1784396/4015115466.py:5: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


In [218]:
output_files= '../output_files/'

In [219]:
# Adjust the path to your CSV file
school_reopen_list_file = '../input_files/11001_schools_open_gps_2021.csv'

# Read the CSV file into a DataFrame
school_reopen_df = pd.read_csv(school_reopen_list_file)


# Ensure 'start_date' and 'end_date' are in the correct datetime format
school_reopen_df['start_date'] = pd.to_datetime(school_reopen_df['start_date'])
school_reopen_df['end_date'] = pd.to_datetime(school_reopen_df['end_date'])

# Replace NaN in zipcode with ""
school_reopen_df['zipcode'] = school_reopen_df['zipcode'].fillna("11001")

school_reopen_df['InPerson'] = school_reopen_df['InPerson'].fillna(0)
school_reopen_df = school_reopen_df.groupby(by=['Grade', 'zipcode', 'start_date', 'end_date']).sum().reset_index()
school_reopen_df['Capacity'] = school_reopen_df['InPerson']  / school_reopen_df['Total_students'] 

school_reopen_df = school_reopen_df[school_reopen_df.Capacity != 0].copy()
del school_reopen_df['CODE']

# Define MinAge and MaxAge based on Grade
school_reopen_df['MinAge'] = np.where(school_reopen_df['Grade'] == 'PREK', 0,
                                      np.where(school_reopen_df['Grade'] == 'PRIMARY', 6,
                                               np.where(school_reopen_df['Grade'] == 'SECONDARY', 11, 18)))

school_reopen_df['MaxAge'] = np.where(school_reopen_df['Grade'] == 'PREK', 5,
                                      np.where(school_reopen_df['Grade'] == 'PRIMARY', 10,
                                               np.where(school_reopen_df['Grade'] == 'SECONDARY', 17, 20)))

# Convert start_date and end_date to numeric days since sim_start_date
sim_start_date = pd.to_datetime('2020-01-01')  # Replace with your simulation start date
school_reopen_df['start_date'] = pd.to_datetime(school_reopen_df['start_date'])
school_reopen_df['end_date'] = pd.to_datetime(school_reopen_df['end_date'])

school_reopen_df['start_day'] = (school_reopen_df['start_date'] - sim_start_date).dt.days
school_reopen_df['end_day'] = (school_reopen_df['end_date'] - sim_start_date).dt.days

# Adjust end_day based on condition
numDays = school_reopen_df['end_day'].max()  # Assuming numDays is the max end_day in the dataset
school_reopen_df['end_day'] = np.where(school_reopen_df['end_day'] == numDays, numDays, school_reopen_df['end_day'])

# Filter out rows where zipcode is "Bogota"
school_reopen_df = school_reopen_df[school_reopen_df['zipcode'] != "Bogota"]

In [220]:
departmento_code = 15

In [221]:
df_schools = pd.read_csv('/zine/HPC02S1/ex-dveloza/AGORA/apps/synthetic_populations/output/formatted_populations/colombia_15/colombia_15_schools.txt', sep=',')

# Correctly transform df_schools to align with grade categories and calculate capacities
df_schools_transformed = df_schools.melt(id_vars=['zipcode'], 
                                         value_vars=['prek', 'kinder', 'gr01_gr12', 'ungraded'], 
                                         var_name='Grade', 
                                         value_name='Capacity')

# Map melted 'Grade' to correspond with 'school_reopen_df_bogota' grade categories
df_schools_transformed['Grade'] = df_schools_transformed['Grade'].map({
    'prek': 'PREK',
    'kinder': 'PRIMARY',
    'gr01_gr12': 'SECONDARY',
    'ungraded': 'UNIVERSITY'
})

df_schools_transformed_dates = pd.DataFrame({})
for _, row in school_reopen_df[['start_date', 'end_date']].drop_duplicates().iterrows():
    df_ = df_schools_transformed.copy()
    df_['start_date']   = row['start_date']
    df_['end_date']     = row['end_date']
    df_schools_transformed_dates = pd.concat([df_schools_transformed_dates, df_])

In [222]:
# Aggregate capacities by zipcode, grade, start_date, and end_date
total_capacity_by_zipcode_grade_date = df_schools_transformed_dates.groupby(['zipcode', 'Grade', 'start_date', 'end_date'])['Capacity'].sum().reset_index()

# Initialize a list to hold the estimated in-person data
estimated_inperson_by_zipcode_grade_date = []

# Iterate through each row in the aggregated data
for _, row in total_capacity_by_zipcode_grade_date.iterrows():
    zipcode = row['zipcode']
    grade = row['Grade']
    start_date = row['start_date']
    end_date = row['end_date']
    capacity = row['Capacity']

    # Filter the dataset for the same grade and date range
    subset = school_reopen_df[(school_reopen_df['Grade'] == grade) &
                              (school_reopen_df['start_date'] <= end_date) &
                              (school_reopen_df['end_date'] >= start_date)].copy()

    if not subset.empty:
        # Normalize capacities
        max_capacity = subset['Total_students'].max()
        normalized_capacity = capacity / max_capacity if max_capacity > 0 else 0
        
        # Calculate the difference in normalized capacity to find the closest match
        subset['NormalizedCapacityDifference'] = (subset['Total_students'] - capacity).abs()
        closest_match = subset.loc[subset['NormalizedCapacityDifference'].idxmin()]
        
        # Use the in-person rate from the closest match
        inperson_rate = closest_match['InPerson'] * normalized_capacity
    else:
        # If no match found, default to a conservative estimate or another logic
        inperson_rate = 0  # Adjust this based on your requirements
    
    # Estimate in-person attendance for the given parameters
    estimated_inperson = inperson_rate
    
    estimated_inperson_by_zipcode_grade_date.append({
        'zipcode': zipcode,
        'Grade': grade,
        'start_date': start_date,
        'end_date': end_date,
        'InPerson': estimated_inperson
    })

# Convert to DataFrame
estimated_inperson_zipcode_date_df = pd.DataFrame(estimated_inperson_by_zipcode_grade_date)

In [223]:
df_estimate = estimated_inperson_zipcode_date_df.merge(total_capacity_by_zipcode_grade_date, on=['zipcode', 'Grade', 'start_date', 'end_date'], how='left').rename(columns={'Capacity' : 'Total_students'})
df_estimate['Capacity'] = (df_estimate['InPerson'] / df_estimate['Total_students']).fillna(0)

In [224]:
df_estimate[df_estimate.Capacity != 0].drop_duplicates().to_csv(f'../output_files/{departmento_code}_schools_open_2021.csv', index=False)

In [225]:
school_reopen_df

,Grade,zipcode,start_date,end_date,InPerson,Total_students,Capacity,MinAge,MaxAge,start_day,end_day
46,PREK,001109,2021-03-05,2021-03-12,7.4,312,0.023718,0,5,429,436
47,PREK,001109,2021-03-12,2021-03-19,7.4,312,0.023718,0,5,436,443
48,PREK,001109,2021-03-19,2021-03-26,7.4,312,0.023718,0,5,443,450
49,PREK,001109,2021-03-26,2021-04-09,26.0,312,0.083333,0,5,450,464
50,PREK,001109,2021-04-09,2021-04-18,10.6,312,0.033974,0,5,464,473
...,...,...,...,...,...,...,...,...,...,...,...
19578,UNIVERSITY,11001,2021-03-19,2021-03-26,16980.5,771034,0.022023,18,20,443,450
19579,UNIVERSITY,11001,2021-03-26,2021-04-09,17129.0,771034,0.022216,18,20,450,464
19580,UNIVERSITY,11001,2021-04-09,2021-04-18,16863.0,771034,0.021871,18,20,464,473
19581,UNIVERSITY,11001,2021-04-18,2021-04-26,16863.0,771034,0.021871,18,20,473,481


# Close data

In [226]:
school_close_df_bogota = pd.read_csv('../input_files/11001_schools_open_gps.csv').melt(id_vars=['LOCALIDAD', 'TOTAL'], 
                                         value_vars=['basic_ed', 'ungraded'], 
                                         var_name='Grade', 
                                         value_name='Total_students').rename(columns={'TOTAL':'InPerson'})

In [227]:
school_close_df_bogota['Grade'] = school_close_df_bogota['Grade'].str.replace('basic_ed', 'SECONDARY').str.replace('ungraded', 'UNIVERSITY')

In [228]:
total_capacity_by_localidad_grade

,zipcode,Grade,Capacity
0,15001,PREK,0
1,15001,PRIMARY,0
2,15001,SECONDARY,43543
3,15001,UNIVERSITY,38312
4,15022,PREK,0
...,...,...,...
487,15879,UNIVERSITY,0
488,15897,PREK,0
489,15897,PRIMARY,0
490,15897,SECONDARY,3168


In [229]:
total_capacity_by_localidad_grade = df_schools_transformed.groupby(['zipcode', 'Grade'])['Capacity'].sum().reset_index()

# Initialize a list to hold the estimated in-person data
estimated_inperson_by_grade = []

# Iterate through each row in the aggregated department data
for _, row in total_capacity_by_localidad_grade.iterrows():
    zipcode = row['zipcode']  # Using 'zipcode' as 'Localidad_ID' from department data
    grade = row['Grade']
    capacity = row['Capacity']

    # Filter the Bogota dataset for the same Localidad_ID and Grade
    subset = school_close_df_bogota[(school_close_df_bogota['Grade'] == grade)].copy()

    if not subset.empty:
        # Normalize capacities based on PropOpen (as a proxy to in-person potential)
        max_prop_open = subset['Total_students'].max()
        normalized_capacity = capacity / max_prop_open if max_prop_open > 0 else 0
        
        # Calculate the difference in normalized capacity to find the closest match
        subset['NormalizedCapacityDifference'] = (subset['Total_students'] - capacity).abs()
        closest_match = subset.loc[subset['NormalizedCapacityDifference'].idxmin()]

        # Use the in-person rate from the closest match
        estimated_inperson = closest_match['InPerson'] * normalized_capacity
    else:
        # If no match found, default to a conservative estimate or another logic
        estimated_inperson = 0  # Adjust based on your requirements or available data
    
    # Append the estimated in-person attendance for the given Localidad and Grade
    estimated_inperson_by_grade.append({
        'zipcode': zipcode,
        'Grade': grade,
        'InPerson': estimated_inperson
    })

# Convert to DataFrame
estimated_inperson_df = pd.DataFrame(estimated_inperson_by_grade)

# Display the DataFrame
print(estimated_inperson_df)


     zipcode       Grade   InPerson
0      15001        PREK   0.000000
1      15001     PRIMARY   0.000000
2      15001   SECONDARY  48.989585
3      15001  UNIVERSITY   1.771696
4      15022        PREK   0.000000
..       ...         ...        ...
487    15879  UNIVERSITY   0.000000
488    15897        PREK   0.000000
489    15897     PRIMARY   0.000000
490    15897   SECONDARY   0.000000
491    15897  UNIVERSITY   0.000000

[492 rows x 3 columns]


In [230]:
df_estimate = estimated_inperson_df.merge(total_capacity_by_localidad_grade, on=['zipcode', 'Grade'], how='left').rename(columns={'Capacity' : 'Total_students'})
df_estimate['Capacity'] = (df_estimate['InPerson'] / df_estimate['Total_students']).fillna(0)

In [231]:
df_estimate[df_estimate.Capacity != 0].drop_duplicates().to_csv(f'../output_files/{departmento_code}_schools_open.csv', index=False)